In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir(os.path.join(os.getcwd(),".."))

import numpy as np
from utils.functions import *
from scipy.signal import hann
import matplotlib.pyplot as plt
%matplotlib widget
import IPython.display as ipd

# Μέρος 1 

## Δημιουργία ημιτονικού σήματος

In [3]:
rate = 44100 #44.1 khz
duration =5 # in sec
time = np.linspace(0,duration, num=rate*duration)
f_s = 1e3 #1kHz
s_m = 1 #Amplitude is 1
s = s_m*np.sin(2*np.pi*f_s*time)

Η παραθύρωση με Hanning/Hamming window εξασφαλίζει σωστή απεικόνιση του φάσματος για οποιοδήποτε αριθμό δειγμάτων (μήκος) του σήματος. Χωρίς το παράθυρο, η διακοπή ενός σήματος με τυχαίο πλάτος οδηγεί στην παραγωγή αρμονικών λόγω της απότομης μετάβασης από την στιγμιαία τιμή του σήματος στο μηδέν. Αντιθέτως, με την παραθύρωση επιτυγχάνεται σταδιακή υποβάθμιση του πλάτους του σήματος προς το μηδέν και έτσι στο φάσμα δεν εισάγονται παραμορφώσεις. Το ίδιο ισχύει και για την εκκίνηση του σήματος.

In [4]:
w = hann(rate*duration)
y = s*w

In [5]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.plot(time[50000:51000], y[50000:51000])
ax.set_title("Sample of a sinusoid 1kHz")
ax.set_xlabel("time (s)")
ax.set_ylabel("Amplitude (V)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Amplitude (V)')

Με τη χρήση του παρακάτω widget μπορείτε να ακούσετε το αρχείο που δημιουργήθηκε.

In [6]:
ipd.display(ipd.Audio(data=s, rate=rate))

Κβάντιση του σήματος με 16 bit και 8 bit αντίστοιχα.

## Κβάντιση του σήματος με 16 bit και 8 bit

In [7]:
yq1 = quantise_audio(y,16,1,-1,0,0)

In [8]:
yq2 = quantise_audio(y,8,1,-1,0,0)

## Απεικόνιση φάσματος χωρίς στρογγυλοποίηση ή μορφοποίηση θορύβου

Φασματική απεικόνιση των δύο σημάτων μετά την κβάντιση.

In [9]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax = plotSpectrum(yq1, rate, ax=ax)
ax = plotSpectrum(yq2, rate, ax=ax)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('SNR (dB)')
ax.set_ylim([-200,5])
ax.set_xlim([0,22050])
ax.set_title('No Dither or Noise Shaping')
ax.legend(['$N = 16$', '$N = 8$'])
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Μέρος 2

## Μείωση σήματος κατά 80 dB

In [10]:
def change_by_dB(s_m,dB):
    """
    Function which accepts initial signal amplitude and returns the amplitude after dB addition 
    or substraction. Power denominator is 20, which is applicable for audio 
    s_m : signal magnitude or numpy array of signal
    dB : decibel change
    return:
        modified signal by dB
    """ 
    power_amplitude_ratio = s_m*10**(dB/20)
    return power_amplitude_ratio

Μείωση του πλάτους του σήματος κατά 80 dB.

In [11]:
y_reduced = change_by_dB(y,-80)

## Κβάντιση μειωμένου σήματος χωρίς την χρήση *dithering*.

In [12]:
yq_16 = quantise_audio(y_reduced,16,1,-1,0,0) # 16 bit quantization
yq_8 = quantise_audio(y_reduced,8,1,-1,0,0) # 8 bit quantization

In [13]:
fig2, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.cla()
ax = plotSpectrum(yq_16, rate, ax=ax)
ax = plotSpectrum(yq_8, rate, ax=ax)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('SNR (dB)')
ax.set_ylim([-200,5])
ax.set_xlim([0,22050])
ax.set_title('Reduced sinusoid\nNo Dither or Noise Shaping')
ax.grid()
ax.legend(['$N = 16$', '$N = 8$'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/user/Documents/jupyter_notebooks/DigitalAudioProcessingExercises/utils/functions.py:170: RuntimeWarning: invalid value encountered in true_divide
  magSpec_dB = 20*np.log10(magSpec/np.max(magSpec))


## Κβάντιση μειωμένου σήματος με την χρήση *dithering*.

In [14]:
yq_16_dith = quantise_audio(y_reduced,16,1,-1,0,1) # 16 bit quantization
yq_8_dith = quantise_audio(y_reduced,8,1,-1,0,1) # 8 bit quantization

In [15]:
fig3, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.cla()
ax = plotSpectrum(yq_16_dith, rate, ax=ax)
ax = plotSpectrum(yq_8_dith, rate, ax=ax)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('SNR (dB)')
ax.set_ylim([-100,5])
ax.set_xlim([0,22050])
ax.set_title('80 dB Reduced sinusoid\nTPDF Dither, No Noise Shaping')
ax.grid()
ax.legend(['$N = 16$', '$N = 8$'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Κβάντιση μειωμένου σήματος με την χρήση *dithering* και *Noise-shaping*.

In [16]:
yq_16_dith_ns = quantise_audio(y_reduced,16,1,-1,2,1) # 16 bit quantization
yq_8_dith_ns = quantise_audio(y_reduced,8,1,-1,2,1) # 8 bit quantization

In [17]:
fig4, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.cla()
ax = plotSpectrum(yq_16_dith_ns, rate, ax=ax)
ax = plotSpectrum(yq_8_dith_ns, rate, ax=ax)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('SNR (dB)')
ax.set_ylim([-80,5])
ax.set_xlim([0,22050])
ax.set_title('80 dB Reduced sinusoid\nTPDF Dither, 2nd order Noise-Shaping')
ax.grid()
ax.legend(['$N = 16$', '$N = 8$'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …